# redo Part III

The assignment asks to produce route info, but in instructor's new book, he wants to produce route transfer info. So I redo this part.

In [1]:
import re; import requests

url = "http://bj.bendibao.com/ditie/"
lines = re.findall('<a style="color:#.+?;" href="/ditie/(.+?)" target="_blank">', requests.get(url+"linemap.shtml").text)
line_names = re.findall('target="_blank">北京地铁(.*?)线路图</a></strong>', requests.get(url+"linemap.shtml").text)
line_names = [s.split("(")[0] for s in line_names]
tmp = zip(lines, line_names)

In [2]:
# complete_line = []
# for s in tmp:
#     complete_line.append(s)
complete_line = [s for s in tmp]
del complete_line[2]
print(complete_line)

[('map_197.shtml', '1号线'), ('map_211.shtml', '2号线'), ('map_198.shtml', '4号线'), ('map_199.shtml', '5号线'), ('map_209.shtml', '6号线'), ('map_253.shtml', '7号线'), ('map_200.shtml', '8号线'), ('map_201.shtml', '9号线'), ('map_212.shtml', '10号线'), ('map_202.shtml', '13号线'), ('map_254.shtml', '14号线东段'), ('map_210.shtml', '14号线西段'), ('map_203.shtml', '15号线'), ('map_204.shtml', '八通线'), ('map_205.shtml', '昌平线'), ('map_207.shtml', '亦庄线'), ('map_206.shtml', '房山线'), ('map_208.shtml', '机场线')]


In [3]:
for s in complete_line:
    print(s[0])

map_197.shtml
map_211.shtml
map_198.shtml
map_199.shtml
map_209.shtml
map_253.shtml
map_200.shtml
map_201.shtml
map_212.shtml
map_202.shtml
map_254.shtml
map_210.shtml
map_203.shtml
map_204.shtml
map_205.shtml
map_207.shtml
map_206.shtml
map_208.shtml


In [4]:
station_per_line = dict() # list of stations per line w/o coordinate
stations = set() # list of stations

station_with_change = r'<a class="link" href="/ditie/zd_.+?\.shtml">(.+?)$'
station_without_change = '<a href="/ditie/zd_.+?\.shtml" class="link" target="_blank">(.+?)$'
for line in complete_line:
    # the order of station on each line does matter
    order = []
    for text in requests.get(url+line[0]).text.split("</a>"):
        if re.search(station_with_change, text):
            stations.update(re.findall(station_with_change, text))# order doesn't matter
            order.append(re.findall(station_with_change, text)[0])
        elif re.search(station_without_change, text):
            stations.update(re.findall(station_without_change, text))# order doesn't matter
            order.append(re.findall(station_without_change, text)[0])
        else: continue
    # I have to write a loop becuase stations with change lanes have different patterns
    station_per_line[line[1]] = order
    
station_per_line['2号线'].append(station_per_line['2号线'][0])
station_per_line['10号线'].append(station_per_line['10号线'][0])
print(station_per_line,'\n\n',stations)

{'1号线': ['苹果园', '古城', '八角游乐园', '八宝山', '玉泉路', '五棵松', '万寿路', '公主坟', '军事博物馆', '木樨地', '南礼士路', '复兴门', '西单', '天安门西', '天安门东', '王府井', '东单', '建国门', '永安里', '国贸', '大望路', '四惠', '四惠东'], '2号线': ['积水潭', '鼓楼大街', '安定门', '雍和宫', '东直门', '东四十条', '朝阳门', '建国门', '北京站', '崇文门', '前门', '和平门', '宣武门', '长椿街', '复兴门', '阜成门', '车公庄', '西直门', '积水潭'], '4号线': ['安河桥北', '北宫门', '西苑', '圆明园', '北京大学东门', '中关村', '海淀黄庄', '人民大学', '魏公村', '国家图书馆', '动物园', '西直门', '新街口', '平安里', '西四', '灵境胡同', '西单', '宣武门', '菜市口', '陶然亭', '北京南站', '马家堡', '角门西', '公益西桥', '新宫', '西红门', '高米店北', '高米店南', '枣园', '清源路', '黄村西大街', '黄村火车站', '义和庄', '生物医药基地', '天宫院'], '5号线': ['宋家庄', '刘家窑', '蒲黄榆', '天坛东门', '磁器口', '崇文门', '东单', '灯市口', '东四', '张自忠路', '北新桥', '雍和宫', '和平里北街', '和平西桥', '惠新西街南口', '惠新西街北口', '大屯路东', '北苑路北', '立水桥南', '立水桥', '天通苑南', '天通苑', '天通苑北'], '6号线': ['海淀五路居', '慈寿寺', '花园桥', '白石桥南', '车公庄西', '车公庄', '平安里', '北海北', '南锣鼓巷', '东四', '朝阳门', '东大桥', '呼家楼', '金台路', '十里堡', '青年路', '褡裢坡', '黄渠', '常营', '草房', '物资学院路', '通州北关', '北运河西', '郝家府', '东夏园', '潞城'], '7号线': ['北京西站', '湾子', '达官营', '广安门内',

In [5]:
connections = dict() # empty dictionary to store connections

for s in stations:
    connections[s] = set()

for key in station_per_line.keys(): # generate real station network
    for i in range(len(station_per_line[key])):
        if i == 0:
            connections[station_per_line[key][i]].add(station_per_line[key][i+1])
        elif i == len(station_per_line[key])-1:
            connections[station_per_line[key][i]].add(station_per_line[key][i-1])
        else:
            connections[station_per_line[key][i]].add(station_per_line[key][i-1])
            connections[station_per_line[key][i]].add(station_per_line[key][i+1])
            
connections

{'东夏园': {'潞城', '郝家府'},
 '育知路': {'平西府', '朱辛庄'},
 '五棵松': {'万寿路', '玉泉路'},
 '雍和宫': {'东直门', '北新桥', '和平里北街', '安定门'},
 '良乡南关': {'良乡大学城西', '苏庄'},
 '北苑': {'望京西', '立水桥'},
 '宣武门': {'和平门', '菜市口', '西单', '长椿街'},
 '泥洼': {'丰台站', '西局'},
 '经海路': {'同济南路', '次渠南'},
 '阜通': {'望京', '望京南'},
 '公主坟': {'万寿路', '军事博物馆', '莲花桥', '西钓鱼台'},
 '上地': {'五道口', '西二旗'},
 '朝阳门': {'东四', '东四十条', '东大桥', '建国门'},
 '褡裢坡': {'青年路', '黄渠'},
 '长阳': {'稻田', '篱笆房'},
 '桥湾': {'珠市口', '磁器口'},
 '龙泽': {'回龙观', '西二旗'},
 '大葆台': {'稻田', '郭公庄'},
 '大屯路东': {'北苑路北', '惠新西街北口'},
 '车公庄西': {'白石桥南', '车公庄'},
 '森林公园南门': {'奥林匹克公园', '林萃桥'},
 '安华桥': {'北土城', '鼓楼大街'},
 '科怡路': {'丰台南路', '丰台科技园'},
 '高米店北': {'西红门', '高米店南'},
 '天宫院': {'生物医药基地'},
 '苏州街': {'巴沟', '海淀黄庄'},
 '团结湖': {'农业展览馆', '呼家楼'},
 '西单': {'复兴门', '天安门西', '宣武门', '灵境胡同'},
 '荣昌东街': {'同济南路', '荣京东街'},
 '白堆子': {'军事博物馆', '白石桥南'},
 '永泰庄': {'林萃桥', '西小口'},
 '郭公庄': {'丰台科技园', '大葆台'},
 '后沙峪': {'南法信', '花梨坎'},
 'T2航站楼': {'T3航站楼'},
 '达官营': {'广安门内', '湾子'},
 '欢乐谷景区': {'南楼梓庄', '垡头'},
 '宋家庄': {'刘家窑', '成寿寺', '石榴庄', '肖村'},
 '南锣鼓巷': 

In [6]:
line_change = {}
for s in stations:
    line_change[s] = set()
for line, stations in station_per_line.items():
    for s in stations:
        line_change[s].add(line)
        
line_change

{'东夏园': {'6号线'},
 '育知路': {'8号线'},
 '五棵松': {'1号线'},
 '雍和宫': {'2号线', '5号线'},
 '良乡南关': {'房山线'},
 '北苑': {'13号线'},
 '宣武门': {'2号线', '4号线'},
 '泥洼': {'10号线'},
 '经海路': {'亦庄线'},
 '阜通': {'14号线东段'},
 '公主坟': {'10号线', '1号线'},
 '上地': {'13号线'},
 '朝阳门': {'2号线', '6号线'},
 '褡裢坡': {'6号线'},
 '长阳': {'房山线'},
 '桥湾': {'7号线'},
 '龙泽': {'13号线'},
 '大葆台': {'房山线'},
 '大屯路东': {'5号线'},
 '车公庄西': {'6号线'},
 '森林公园南门': {'8号线'},
 '安华桥': {'8号线'},
 '科怡路': {'9号线'},
 '高米店北': {'4号线'},
 '天宫院': {'4号线'},
 '苏州街': {'10号线'},
 '团结湖': {'10号线'},
 '西单': {'1号线', '4号线'},
 '荣昌东街': {'亦庄线'},
 '白堆子': {'9号线'},
 '永泰庄': {'8号线'},
 '郭公庄': {'9号线', '房山线'},
 '后沙峪': {'15号线'},
 'T2航站楼': {'机场线'},
 '达官营': {'7号线'},
 '欢乐谷景区': {'7号线'},
 '宋家庄': {'10号线', '5号线', '亦庄线'},
 '南锣鼓巷': {'6号线', '8号线'},
 '广渠门内': {'7号线'},
 '清华东路西口': {'15号线'},
 '俸伯': {'15号线'},
 '临河里': {'八通线'},
 '枣营': {'14号线东段'},
 '潘家园': {'10号线'},
 '义和庄': {'4号线'},
 '马家堡': {'4号线'},
 '大望路': {'14号线东段', '1号线'},
 '永定门外': {'14号线东段'},
 '北新桥': {'5号线'},
 '复兴门': {'1号线', '2号线'},
 '郝家府': {'6号线'},
 '军事博物馆': {'1号线', '9号线'}

In [7]:
def change_routes(path):
    change = []
    for i in range(len(path)-2):
        change_1 = line_change[path[i+1]] - (line_change[path[i+1]] - line_change[path[i]])
        change_2 = line_change[path[i+2]] - (line_change[path[i+2]] - line_change[path[i+1]])
        # https://stackoverflow.com/questions/2864842/common-elements-comparison-between-2-lists
        if change_1 == change_2: continue
        else:
            node = '在'+change_1.pop()+path[i]+'站转'+change_2.pop()
            change.append(node)
    change = "; ".join(change)
    return change

In [8]:
def change_num(pathes):
    num = []
    for path in pathes:
        j = 0
        for i in range(len(path)-2):
            change_1 = line_change[path[i+1]] - (line_change[path[i+1]] - line_change[path[i]])
            change_2 = line_change[path[i+2]] - (line_change[path[i+2]] - line_change[path[i+1]])
            if change_1 == change_2: continue
            else:
                j += 1
        num.append(j)
    return num

In [9]:
def search(start, destination):
    pathes = [[start]]
    visitied = set()
    while pathes:
        path = pathes.pop(0)
        frontier = path[-1]
        if frontier in visitied: continue
        successors = connections[frontier]
        for step in successors:
            if step in path: continue
            new_path = path + [step]
            pathes.append(new_path)
            if step == destination: 
                route = f"从{start}上车, "+change_routes(new_path)+f", 到{destination}下车"
                return route
        visitied.add(frontier)
        pathes = sorted(pathes, key=lambda x: change_num(pathes))
        
print(search('西直门', '国展'))
print(search('西直门', '西单'))

从西直门上车, 在2号线积水潭站转8号线; 在8号线安华桥站转10号线; 在10号线惠新西街南口站转13号线; 在13号线芍药居站转15号线, 到国展下车
从西直门上车, 在2号线阜成门站转1号线, 到西单下车
